# Mount Google Drive

Mount your google drive to be used for storing the storing the dataset into it (optional to save datasets on mega to it) and also for saving the results in the drive.

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

# Move Dataset from Mega to Drive

Move the file with the provided link (Provide your username and password if it's not a public link) to your google drive already mounted before in the previous step. 

### Install Mega

In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)


if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()


### provide URL and Mega ID

Add the URL to fetch from Mega, and the username and password for those files if it wasn't a public URL and to use the pro mega quota if you have a pro account.

In [ ]:
URL = "" 
OUTPUT_PATH = "" 
USERNAME = ""  
PASSWORD = "" 

In [ ]:
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        

if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"


class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


@timeout(10)
def runShT(args):
    return runSh(args, output=True)

def login(): 
    runShT(f"mega-login {USERNAME} {PASSWORD}")
    
if not (USERNAME == "" or PASSWORD == ""):
    try:
        login()
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")

transfare()

# Fetch & Clone Repo

Clone the `image-tagging-tools` repo as it has all the required utils and codes from preprocessing the image datasets to training the models and using them to classify the images. 

In [ ]:
!git clone https://github.com/kk-digital/image-tagging-tools.git

# Preprocess the dataset images (Stage 1)

Use the `image-dataset-processor` utils from the previously cloned repo to process a directory of images (paths to directory of images or an archived dataset), and computes the images metadata along with its CLIP embeddings and writes the result into a JSON file into `output_folder`

### Install requirements 

In [ ]:
%pip install -r  ./image-tagging-tools/image-dataset-processor/requirements.txt

### Import the module/Utility 

In [ ]:
import sys
sys.path.insert(0, './image-tagging-tools/image-dataset-processor/')

from ImageDatasetProcessor import ImageDatasetProcessor

### set required variables by the utility

Initialize the required parameters needed by the dataset preprocessor utility and they are described as follows: 

* `input_folder` _[str]_ -  path to the directory containing sub-folders of each tag.
* `output_folder` _[str]_ - path to the directory where to save the files into it.

* `clip_model` _[str]_ - CLIP model to be used

* `pretrained` _[str]_ - the pre-trained model to be used for CLIP
* `batch_size` _[int]_ -  number of images to process at a time
* `num_threads` _[int]_ - the number to be used in this process

* `device` _[str]_ -  the device to be used in computing the CLIP embeddings, if `None` is provided then `cuda` will be used if available


In [ ]:
dataset_path = ""
output_folder = ""
clip_model = "ViT-B-32"
pretrained = 'openai'
batch_size = 32,
num_threads = 4,
device = None, 

### Run the Preprocessor

In [ ]:
ImageDatasetProcessor.process_dataset(
    dataset_path, 
    output_folder, 
    clip_model, 
    pretrained, 
    batch_size, 
    num_threads, 
    device
)